In [1]:
import pymssql
import _mssql
import datetime
from decimal import Decimal

In [2]:
pymssql.__version__

'2.2.0'

In [3]:
server = 'h9ggwlagd1.database.windows.net'
user = 'modelread'
password = '4fSHRXwd4u'

In [4]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['tmldb']

In [5]:
def convert_dates(values):
    k,v = values
    if type(v) == datetime.date:
        return k, datetime.datetime.combine(
                v,
                datetime.datetime.min.time())
    if type(v) == Decimal:
        return k, float(v)
    else:
        return k, v
    
d = {'a': datetime.date.min, 'b': 'adadad'}

In [6]:
conn = pymssql.connect(server, user+"@"+server, password, "TMLDB")

In [8]:
c2 = conn.cursor(as_dict=True)

In [15]:
colname = 'contracts'
c2.execute('SELECT * FROM cqgdb.tbl'+colname)

In [16]:
collection = db[colname]

In [17]:
cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))


Done 6050 rows


In [11]:
pymongo.__version__

'3.2.2'

In [ ]:
c2.rowcount

In [18]:
import sys
sys.path.append('/home/ubertrader/Dropbox/tmqrexo/tmqrv')
sys.path.append('/home/ubertrader/Dropbox/tmqrexo/tmqrv/exobuilder/tests')

In [22]:
from exobuilder.contracts.futureschain import FuturesChain
from exobuilder.contracts.futurecontract import FutureContract
from exobuilder.tests.assetindexdict import AssetIndexDicts
from datetime import datetime, date
from exobuilder.contracts.instrument import Instrument
from exobuilder.data.datasource_mongo import DataSourceMongo
from exobuilder.data.assetindex_mongo import AssetIndexMongo

In [21]:
assetindex = AssetIndexDicts()
symbol = 'EP'
date = datetime(2014, 1, 6, 10, 15, 0)
futures_limit = 3
instrument = Instrument(assetindex, symbol, date, futures_limit, options_limit=10)
fut_chain = FuturesChain(instrument)

AttributeError: 'AssetIndexDicts' object has no attribute 'assetindex'

In [23]:
mongo_connstr = 'mongodb://localhost:27017/'
mongo_db_name = 'tmldb'
assetindex = AssetIndexMongo(mongo_connstr, mongo_db_name)
date = datetime(2014, 1, 6, 10, 15, 0)
futures_limit = 3
options_limit = 10

datasource = DataSourceMongo(mongo_connstr, mongo_db_name, assetindex, date, futures_limit, options_limit)

In [24]:
fut_id = datasource['ES'].futures[0].dbid

In [25]:
c2 = conn.cursor(as_dict=True)
c2.execute("SELECT * FROM cqgdb.tblbardata where idcontract = '{0}'".format(fut_id))
collection = db['futures_data']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))

Done 116046 rows


In [11]:
# Real price on ("2014-01-06 10:15:00")
instrument.futures[0]._price = 1819.5

In [26]:
opt_chain = datasource['ES'].futures[0].options[0]

In [27]:
options_dbids = []
for optpair in opt_chain:
    options_dbids.append(optpair.C.dbid)
    options_dbids.append(optpair.P.dbid)

In [28]:
tuple(options_dbids[0:2])

(11488838, 11489000)

In [ ]:
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tbloptiondata where idoption in {0}".format(tuple(options_dbids))
print(query)
c2.execute(query)
collection = db['options_data']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break

print('Done {0} rows'.format(cnt))

In [31]:
row

{'idoptioninputdata': 1116016,
 'idoptioninputsymbol': 15,
 'optioninputclose': 0.81,
 'optioninputdatetime': datetime.datetime(2016, 7, 21, 0, 0),
 'optioninputhigh': 0.775,
 'optioninputlow': 0.815,
 'optioninputopen': 0.785}

## RFR

In [30]:
c2 = conn.cursor(as_dict=True)
query = "SELECT * FROM cqgdb.tbloptioninputdata"
print(query)
c2.execute(query)

collection = db['options_data_inputs']

cnt = 0
for row in c2:
    try:
        data = dict(map(convert_dates, row.items()))
        collection.insert_one(data)    
        cnt += 1
    except TypeError:
        print('TypeError')
        print(row)
        break
print('Done {0} rows'.format(cnt))

SELECT * FROM cqgdb.tbloptioninputdata
Done 3086 rows


In [1]:
from notebook.auth import passwd

In [2]:
passwd()

Enter password: ········
Verify password: ········


'sha1:6a3647e6f696:9abcbf34f0f0943b79fde23ea33d6ae7fa43b1ac'